# Phase 2: Semantic Retrieval with FAISS



## Step 0 — Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install sentence-transformers faiss-cpu pandas numpy tqdm -q
print('✅ Dependencies installed.')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ Dependencies installed.


## Step 1 — Load Products from Phase 1

In [2]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('data')

df = pd.read_csv(DATA_DIR / 'products_clean.csv')

# Drop rows with missing descriptions (safety check)
df = df.dropna(subset=['description']).reset_index(drop=True)

print(f'✅ Loaded {len(df):,} products')
print(f'Columns: {df.columns.tolist()}')
df.head(3)

✅ Loaded 10,000 products
Columns: ['asin', 'title', 'description', 'category', 'price', 'rating', 'rating_count', 'store', 'main_image']


,asin,title,description,category,price,rating,rating_count,store,main_image
0,B07R3DYMH6,Set of 4 Irish Coffee Glass Mugs Footed 10.5 o...,Set of 4 Irish Coffee Glass Mugs Footed 10.5 o...,Home & and & Kitchen,24.95,4.6,18,LavoHome,NaN
1,B0BNZ8Q7YT,Foaming Soap Dispenser Thick Ceramic Foam Hand...,Foaming Soap Dispenser Thick Ceramic Foam Hand...,Home & and & Kitchen,24.99,4.4,135,rejomiik,NaN
2,B00KKU8HTG,jersey seating 2 x Vinyl Air Lift Adjustable S...,jersey seating 2 x Vinyl Air Lift Adjustable S...,Home & and & Kitchen,NaN,4.3,167,jersey seating®,NaN


## Step 2 — Prepare Text for Embedding
We combine `title` + `description` into one string per product.
Title is repeated at the front because it carries the most signal.

In [3]:
def build_embed_text(row):
    """Combine title + description into one rich string for embedding."""
    title = str(row.get('title', '') or '').strip()
    desc  = str(row.get('description', '') or '').strip()
    # Truncate description to 300 chars to avoid token limit issues
    desc  = desc[:300]
    return f"{title}. {desc}"

df['embed_text'] = df.apply(build_embed_text, axis=1)

print(f'✅ Embed text prepared for {len(df):,} products')
print(f'Avg embed text length: {df["embed_text"].str.len().mean():.0f} chars')
print(f'\nSample embed text:')
print(df['embed_text'].iloc[0])

✅ Embed text prepared for 10,000 products
Avg embed text length: 408 chars

Sample embed text:
Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For Coffee, tea, Cappuccinos, Mulled Ciders,Hot Chocolates, Ice cream and More. Set of 4 Irish Coffee Glass Mugs Footed 10.5 oz.Thick Wall Glass For Coffee, tea, Cappuccinos, Mulled Ciders,Hot Chocolates, Ice cream and More ☕PERFECT IRISH COFFEE MUG: With our clear glass 10.5 ounce makes it a perfect mug for offering up your signature Coffee Drinks, Cappuccinos, Milk shakes, De


## Step 3 — Load Sentence Transformer Model
`all-MiniLM-L6-v2` is ~80MB, downloads once and caches locally.
It produces 384-dimensional embeddings and is fast on CPU.

In [4]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = 'all-MiniLM-L6-v2'

print(f'Loading model: {MODEL_NAME}')
print('First run downloads ~80MB — cached after that...')

model = SentenceTransformer(MODEL_NAME)

print(f'✅ Model loaded')
print(f'   Embedding dimension : {model.get_sentence_embedding_dimension()}')
print(f'   Max sequence length : {model.max_seq_length}')

/Users/rithikabaskaran/MyFiles/ShopSense/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model: all-MiniLM-L6-v2
First run downloads ~80MB — cached after that...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2243.62it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Model loaded
   Embedding dimension : 384
   Max sequence length : 256


## Step 4 — Generate Embeddings for All Products
Embeds all 10,000 product descriptions.
Takes ~5-10 minutes on CPU — grab a coffee ☕

In [5]:
import numpy as np
import time

texts = df['embed_text'].tolist()

print(f'Embedding {len(texts):,} product descriptions...')
print('This takes ~5-10 min on CPU. Progress bar below:')

start = time.time()

embeddings = model.encode(
    texts,
    batch_size=64,          # process 64 at a time
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # L2 normalize — makes cosine similarity = dot product
)

elapsed = time.time() - start

print(f'\n✅ Embeddings generated')
print(f'   Shape     : {embeddings.shape}  (products × embedding_dim)')
print(f'   Data type : {embeddings.dtype}')
print(f'   Time taken: {elapsed/60:.1f} min')

Embedding 10,000 product descriptions...
This takes ~5-10 min on CPU. Progress bar below:


Batches: 100%|██████████| 157/157 [00:15<00:00, 10.46it/s]


✅ Embeddings generated
   Shape     : (10000, 384)  (products × embedding_dim)
   Data type : float32
   Time taken: 0.3 min


## Step 5 — Build FAISS Index
FAISS (Facebook AI Similarity Search) stores all embeddings and lets us find the nearest neighbors to any query vector in milliseconds.

We use `IndexFlatIP` — Inner Product index. Since embeddings are L2 normalized, inner product = cosine similarity.

In [6]:
import faiss

EMBEDDING_DIM = embeddings.shape[1]  # 384 for MiniLM

# Build the index
index = faiss.IndexFlatIP(EMBEDDING_DIM)   # IP = Inner Product (cosine sim)
index.add(embeddings.astype('float32'))    # FAISS requires float32

print(f'✅ FAISS index built')
print(f'   Index type       : IndexFlatIP (exact cosine similarity)')
print(f'   Embedding dim    : {EMBEDDING_DIM}')
print(f'   Vectors indexed  : {index.ntotal:,}')

✅ FAISS index built
   Index type       : IndexFlatIP (exact cosine similarity)
   Embedding dim    : 384
   Vectors indexed  : 10,000


## Step 6 — Save FAISS Index + Metadata

In [7]:
import json

# Save FAISS index
index_path = DATA_DIR / 'faiss_index.bin'
faiss.write_index(index, str(index_path))
print(f'✅ FAISS index saved → {index_path} ({index_path.stat().st_size/1024:.0f} KB)')

# Save metadata — maps each FAISS position (0, 1, 2...) to product info
# This is how we go from "index position 42" back to the actual product
metadata = df[['asin', 'title', 'price', 'rating', 'rating_count', 'category', 'description']].to_dict(orient='records')

meta_path = DATA_DIR / 'faiss_metadata.json'
with open(meta_path, 'w') as f:
    json.dump(metadata, f)

print(f'✅ Metadata saved    → {meta_path} ({meta_path.stat().st_size/1024:.0f} KB)')
print(f'\n   Metadata contains {len(metadata):,} product records')
print(f'   Fields: {list(metadata[0].keys())}')

✅ FAISS index saved → data/faiss_index.bin (15000 KB)
✅ Metadata saved    → data/faiss_metadata.json (13319 KB)

   Metadata contains 10,000 product records
   Fields: ['asin', 'title', 'price', 'rating', 'rating_count', 'category', 'description']


## Step 7 — Build the Search Function
This is the core of Phase 2. Takes a natural language query, embeds it, and returns the top-K most similar products.

In [8]:
def search(query: str, top_k: int = 20) -> list[dict]:
    """
    Search for products using natural language.

    Args:
        query  : Natural language search string e.g. 'compact coffee maker for small kitchen'
        top_k  : Number of results to return (default 20, reranker will pick top 5)

    Returns:
        List of dicts with product info + similarity score
    """
    # 1. Embed the query using the same model
    query_embedding = model.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    # 2. Search the FAISS index
    scores, indices = index.search(query_embedding, top_k)

    # 3. Build results list
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:   # FAISS returns -1 for empty slots
            continue
        product = metadata[idx].copy()
        product['similarity_score'] = round(float(score), 4)
        results.append(product)

    return results


print('✅ search() function ready.')
print('Usage: results = search("your query here", top_k=20)')

✅ search() function ready.
Usage: results = search("your query here", top_k=20)


## Step 8 — Test the Search Function
Let's run a few queries and see what comes back!

In [9]:
def display_results(query: str, top_k: int = 5):
    """Pretty print search results."""
    print(f'🔍 Query: "{query}"')
    print('─' * 65)
    results = search(query, top_k=top_k)
    for i, r in enumerate(results, 1):
        price_str = f"${r['price']:.2f}" if r['price'] else 'N/A'
        print(f"{i}. {r['title'][:70]}")
        print(f"   Price: {price_str}  |  Rating: {r['rating']}⭐ ({r['rating_count']:,} reviews)")
        print(f"   Similarity: {r['similarity_score']}")
        print()
    print('=' * 65)

# ── Test Query 1 ──
display_results('compact coffee maker for small kitchen', top_k=5)

🔍 Query: "compact coffee maker for small kitchen"
─────────────────────────────────────────────────────────────────
1. Small Coffee Table White Modern Wood Mini Rectangle Coffee Tables for 
   Price: $52.42  |  Rating: 4.6⭐ (80 reviews)
   Similarity: 0.5822

2. Famiworths Single Serve Coffee Maker for K Cup and Ground Coffee, 6 to
   Price: $nan  |  Rating: 3.9⭐ (246 reviews)
   Similarity: 0.537

3. Famiworths Single Serve Coffee Maker for K Cup and Ground Coffee, 6 to
   Price: $46.18  |  Rating: 4.3⭐ (4,620 reviews)
   Similarity: 0.5354

4. Black & Decker DCM7 Cup-at-a-Time Personal 12-Ounce Coffeemaker, White
   Price: $nan  |  Rating: 3.6⭐ (237 reviews)
   Similarity: 0.5084

5. Home Basics Stainless Steel Stove Top Espresso Coffee Maker (Silver) P
   Price: $15.88  |  Rating: 3.8⭐ (52 reviews)
   Similarity: 0.4936



In [10]:
# ── Test Query 2 ──
display_results('non stick frying pan under $30', top_k=5)

🔍 Query: "non stick frying pan under $30"
─────────────────────────────────────────────────────────────────
1. Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Skillet Small
   Price: $nan  |  Rating: 4.2⭐ (29 reviews)
   Similarity: 0.6034

2. Sakuchi Frying Pan Set Nonstick Skillet Set Induction Compatible 3 Pie
   Price: $49.99  |  Rating: 4.6⭐ (26 reviews)
   Similarity: 0.5792

3. Copper Frying Pan 11-Inch Non Stick Ceramic Infused Titanium Steel Ove
   Price: $16.99  |  Rating: 4.3⭐ (57 reviews)
   Similarity: 0.5718

4. MICHELANGELO Frying Pan with Lid, Nonstick 8 Inch Frying Pan with Cera
   Price: $23.99  |  Rating: 4.4⭐ (10,637 reviews)
   Similarity: 0.5677

5. Nonstick Cookware Set, 20 Piece Tiffany Blue Pots and Pans Set, Cerami
   Price: $89.99  |  Rating: 4.4⭐ (50 reviews)
   Similarity: 0.5548



In [11]:
# ── Test Query 3 ──
display_results('gift for someone who loves cooking', top_k=5)

🔍 Query: "gift for someone who loves cooking"
─────────────────────────────────────────────────────────────────
1. LITSTYLES Charcuterie Boards, Bamboo Cheese Board & Knife Set, Premium
   Price: $20.99  |  Rating: 3.9⭐ (26 reviews)
   Similarity: 0.5198

2. Funny Fork Gifts for Women Men Kids, Cute My Mac and Cheese Fork Engra
   Price: $10.99  |  Rating: 5.0⭐ (11 reviews)
   Similarity: 0.5071

3. Landisun Apron Kitchen Chef Cooking Gag Gift Creative Funny Grilling B
   Price: $14.99  |  Rating: 4.6⭐ (154 reviews)
   Similarity: 0.5005

4. Funny Gifts for Women Wife Girlfriend Friends, 20 oz Wine Tumbler with
   Price: $nan  |  Rating: 4.6⭐ (59 reviews)
   Similarity: 0.4919

5. Cat Measuring Spoons, Cat Shaped Ceramic Measuring Spoon, Cat Mom Gift
   Price: $29.99  |  Rating: 4.5⭐ (250 reviews)
   Similarity: 0.4752



In [12]:
# ── Test Query 4 — try your own! ──
display_results('storage solution for small apartment', top_k=5)

🔍 Query: "storage solution for small apartment"
─────────────────────────────────────────────────────────────────
1. storageLAB Underbed Storage Containers, Storage Bin for Clothes, Blank
   Price: $64.99  |  Rating: 4.7⭐ (12,909 reviews)
   Similarity: 0.5294

2. MAGINELS Portable Storage Cubes-14 x14 Cube (16 Cubes)-More Stable (ad
   Price: $119.99  |  Rating: 4.0⭐ (89 reviews)
   Similarity: 0.4881

3. KUPPET Compact Refrigerator, Mini Fridge with Freezer for Bedroom, Dro
   Price: $nan  |  Rating: 4.1⭐ (18 reviews)
   Similarity: 0.4549

4. GOOD IDEAS. Savannah Column Storage and Waste Bin - Oak
   Price: $nan  |  Rating: 4.3⭐ (94 reviews)
   Similarity: 0.4522

5. Kitchen Organizer Fridge Freezer Storage - 4 Sets x 5L Stackable Large
   Price: $nan  |  Rating: 4.2⭐ (692 reviews)
   Similarity: 0.4518



## Step 9 — Reload Test (verify saved index works)
Simulates loading the index fresh — exactly what Phase 3 will do.

In [13]:
print('Testing reload from disk...')

# Load index from disk
index_loaded = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))

# Load metadata from disk
with open(DATA_DIR / 'faiss_metadata.json') as f:
    metadata_loaded = json.load(f)

print(f'✅ Index reloaded — {index_loaded.ntotal:,} vectors')
print(f'✅ Metadata reloaded — {len(metadata_loaded):,} records')

# Quick sanity search
q_emb = model.encode(['kitchen knife set'], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
scores, indices = index_loaded.search(q_emb, 3)
print(f'\nSanity search → "kitchen knife set":')
for score, idx in zip(scores[0], indices[0]):
    print(f'  [{score:.4f}] {metadata_loaded[idx]["title"][:70]}')

Testing reload from disk...
✅ Index reloaded — 10,000 vectors
✅ Metadata reloaded — 10,000 records

Sanity search → "kitchen knife set":
  [0.7202] Kitchen Chef Knives Set with Wood Grain Handle 6 Pieces Stainless Stee
  [0.7114] Daanaas Knife Set with Wooden Block and Sharpener 16 Pieces,Profession
  [0.7021] Professional Chef Knife Set 5 Pieces, Kitchen Knives Set Stainless Ste


## Step 10 — Final Summary

In [14]:
print('=' * 55)
print('     PHASE 2 COMPLETE — SUMMARY')
print('=' * 55)
print(f'  Model              : {MODEL_NAME}')
print(f'  Embedding dim      : {EMBEDDING_DIM}')
print(f'  Products indexed   : {index.ntotal:,}')
print(f'  Index type         : IndexFlatIP (cosine similarity)')
print()
print('  Files created:')
for fname in ['faiss_index.bin', 'faiss_metadata.json']:
    fpath = DATA_DIR / fname
    print(f'    ✅ data/{fname} ({fpath.stat().st_size/1024:.0f} KB)')
print()
print('  ➡️  Next: Phase 3 — Cross-Encoder Reranking (Colab, GPU)')
print('=' * 55)

     PHASE 2 COMPLETE — SUMMARY
  Model              : all-MiniLM-L6-v2
  Embedding dim      : 384
  Products indexed   : 10,000
  Index type         : IndexFlatIP (cosine similarity)

  Files created:
    ✅ data/faiss_index.bin (15000 KB)
    ✅ data/faiss_metadata.json (13319 KB)

  ➡️  Next: Phase 3 — Cross-Encoder Reranking (Colab, GPU)
